# Heart Failure Prediction Analysis

Heart failure is a common and serious condition that occurs when the heart is unable to pump enough blood to meet the body's needs. Predicting heart failure events can be crucial for early intervention and prevention of adverse outcomes.

Machine learning models can be used to predict heart failure based on clinical data, providing valuable insights for healthcare professionals. These models can analyze various patient factors such

This notebook presents a comprehensive analysis of heart failure prediction using various machine learning models. We'll go through data loading, exploratory data analysis (EDA), feature engineering, model training, and evaluation. We'll compare multiple classification models and identify the best-performing model for predicting heart failure events.

The dataset used in this analysis is the "Heart Failure Clinical Records" dataset, which contains 12 clinical features and a target variable indicating whether a heart failure event occurred. The goal is to build a predictive model that can accurately identify patients at risk of heart failure.

Let's get started with the analysis!

## Overview

This analysis will be divided into the following sections:

1. Import Libraries
2. Load the Data
3. Exploratory Data Analysis (EDA)
4. Feature Engineering
5. Model Preparation
6. Model Definition and Hyperparameter Grids
7. Model Training and Evaluation
8. Results Comparison
9. Best Model Analysis

Let's begin with the first step: importing the necessary libraries.

## 1. Import Libraries

We start by importing all necessary libraries for data manipulation, visualization, and machine learning:

- pandas and numpy for data manipulation
- matplotlib and seaborn for data visualization
- scipy for statistical functions
- sklearn for machine learning models, preprocessing, model selection, and evaluation metrics
- warnings to ignore FutureWarnings


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures,OneHotEncoder
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, precision_recall_curve, average_precision_score

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier

import shap
from sklearn.feature_selection import RFE
import os

# ignore all future warnings
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

## 2. Dataset

Our dataset contains various clinical features and a target variable indicating whether a heart failure event occurred. There are 13 clinical features:

- age: age of the patient (years)
- anaemia: decrease of red blood cells or hemoglobin (boolean)
- creatinine phosphokinase  (CPK): level of the CPK enzyme in the blood (mcg/L)
- diabetes: if the patient has diabetes (boolean)
- ejection fraction: percentage of blood leaving the heart at each contraction  (percentage)
- high blood pressure: if the patient has hypertension (boolean)
- platelets: platelets in the blood (kiloplatelets/mL)
- sex: woman or man (binary)
- serum creatinine: level of serum creatinine in the blood (mg/dL)
- serum sodium: level of serum sodium in the blood (mEq/L)
- smoking: if the patient smokes or not (boolean)
- time: follow-up period (days)
- [target] death event: if the patient died during the follow-up period (boolean)

Here's the reference of the dataset to download the link: https://archive.ics.uci.edu/dataset/519/heart+failure+clinical+records

In [ ]:
# For docker
#data = pd.read_csv('heart_failure_clinical_records_dataset.csv')

# For local
data = pd.read_csv('heart_failure_clinical_records_dataset.csv')
display(data.head())

## 3. Exploratory Data Analysis (EDA)

1. We define a function `improved_eda(data)` to perform comprehensive exploratory data analysis:

    1. Display dataset information (column names, non-null counts, dtypes)
    2. Check for missing values
    3. Show summary statistics (count, mean, std, min, 25%, 50%, 75%, max) for numerical columns
    4. Display the class distribution of the target variable 'DEATH_EVENT'
    5. Create a correlation heatmap to visualize relationships between features
    6. Plot distribution of numerical features, separated by the target variable
    7. Create box plots for numerical features, grouped by the target variable
    8. Detect outliers using the Interquartile Range (IQR) method

2. `plot_feature_distributions(data)`: Creates distribution plots for all features, separated by the target variable.

3. `detect_outliers(data, columns, method='iqr')`: Detects outliers using the Interquartile Range (IQR) method.

This function provides a thorough overview of the dataset's characteristics, helping to inform subsequent preprocessing and modeling decisions.

In [ ]:
def improved_eda(data):
    print("Dataset Information:")
    print(data.info())
    
    print("\nMissing Values:")
    print(data.isnull().sum())
    
    print("\nSummary Statistics:")
    print(data.describe())
    
    print("\nClass Distribution:")
    print(data['DEATH_EVENT'].value_counts(normalize=True))
    
    # Correlation heatmap
    plt.figure(figsize=(12, 10))
    sns.heatmap(data.corr(), annot=True, cmap='coolwarm')
    plt.title('Correlation Heatmap')
    plt.show()
    
    # Distribution plots for numerical features
    num_features = data.select_dtypes(include=[np.number]).columns
    n_features = len(num_features)
    
    # Box plots for numerical features
    fig, axes = plt.subplots(n_features // 3 + 1, 3, figsize=(20, 5 * (n_features // 3 + 1)))
    axes = axes.flatten()
    
    for i, col in enumerate(num_features):
        sns.boxplot(data=data, x='DEATH_EVENT', y=col, ax=axes[i])
        axes[i].set_title(f'Box Plot of {col} by DEATH_EVENT')
    
    plt.tight_layout()
    plt.show()

improved_eda(data)

# Advanced EDA
def plot_feature_distributions(data):
    n_features = len(data.columns)
    fig, axes = plt.subplots(n_features // 3 + 1, 3, figsize=(20, 5 * (n_features // 3 + 1)))
    axes = axes.flatten()
    
    for i, col in enumerate(data.columns):
        sns.histplot(data=data, x=col, hue='DEATH_EVENT', kde=True, ax=axes[i])
        axes[i].set_title(f'Distribution of {col}')
    
    plt.tight_layout()
    plt.show()

plot_feature_distributions(data)
    
# Outlier detection and handling function
def detect_outliers(data, columns, method='iqr'):
    for col in columns:
        if method == 'iqr':
            Q1 = data[col].quantile(0.25)
            Q3 = data[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
        elif method == 'zscore':
            z_scores = np.abs(stats.zscore(data[col]))
            lower_bound = data[col].mean() - 3 * data[col].std()
            upper_bound = data[col].mean() + 3 * data[col].std()
        
        print(f"Outliers in {col}:")
        print(data[(data[col] < lower_bound) | (data[col] > upper_bound)][col])
    
    return data

detect_outliers(data, data.select_dtypes(include=[np.number]).columns, method='iqr')

## 4. Feature Engineering

We define two functions for feature engineering:

1. `one_hot_encoding(df, columns)`: Performs one-hot encoding on specified categorical columns.

2. `feature_engineering(data)`: 
   - Creates an 'age_group' feature and one-hot encodes it
   - Adds an interaction feature between anemia and diabetes
   - Creates a 'risk_factor' feature based on high blood pressure, smoking, and diabetes

These new features aim to capture more complex relationships in the data that might improve model performance.

In [ ]:
def one_hot_encoding(df,columns):
        for column in columns:
            df = pd.concat([df,pd.get_dummies(df[column],prefix=column)],axis=1)
            df.drop(column,axis=1,inplace=True)
        return df

def feature_engineering(data):
    # Add age_group categorical feature
    data['age_group'] = pd.cut(data['age'], bins=[0, 30, 45, 60, 75, 100], labels=['Young', 'Middle-aged', 'Senior', 'Elderly', 'Very Elderly'])

    data = one_hot_encoding(data,['age_group'])

    # Add anemia and diabetes interaction feature
    data['anemia_diabetes_interaction'] = data['anaemia'] * data['diabetes']
    
    # If you are high blood pressure and/or smoke diabetes and senior or elder, you are at higher risk of heart failure. Convert this to a binary feature (1 or 0)
    data['risk_factor'] = ((data['high_blood_pressure'] == 1) | (data['smoking'] == 1)) & ((data['diabetes'] == 1)) 
    data['risk_factor'] = data['risk_factor'].astype(int)

    return data

## 5. Model Preparation

We prepare the data for modeling:

1. Separate features (X) and target variable (y)
2. Apply feature engineering to X
3. Display the first few rows of the engineered feature set
4. Split the data into training and test sets (80% train, 20% test) using stratified sampling

This step ensures our data is ready for model training and evaluation.


In [ ]:
# Separate features and target
X = data.drop(['DEATH_EVENT'], axis=1)
y = data['DEATH_EVENT']

# Feature Engineering
X = feature_engineering(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

## 6. Model Definition and Hyperparameter Grids

This cell defines the models and their hyperparameter search spaces:

1. It creates search spaces for each model's hyperparameters. For example, `search_space_svc` defines different kernels, gamma values, and C values for the Support Vector Classifier.

2. It defines a dictionary `dict_models` that contains:
   - The model object
   - Its corresponding hyperparameter search space
   - A boolean indicating whether the model requires feature scaling
   
The ML algorithms are: 
1. Support Vector Classifier (SVC)
2. Logistic Regression
3. Decision Tree
4. Random Forest
5. Gradient Boosting
6. AdaBoost
7. Bagging
8. Extra Trees
9. K-Nearest Neighbors


This setup allows for systematic comparison of multiple models (SVC, Logistic Regression, Decision Trees, Random Forests, etc.) and hyperparameter tuning using GridSearchCV in the next step.

In [ ]:
search_space_svc = [{'svc__kernel': ['linear', 'poly','rbf'],
                    'svc__gamma': ['scale'],
                    'svc__C': [0.1, 1]}]

search_space_lr = [{'logisticregression__C': [0.1, 1],
                    'logisticregression__penalty': ['l1', 'l2']}]

search_space_ridge = [{'ridge__alpha': [0.1, 1]}]

search_space_decisiontree = [{'decisiontreeclassifier__max_depth': [2, 4, 6]}]
search_space_randomforest = [{'randomforestclassifier__n_estimators': [10,50,100,200],
                                'randomforestclassifier__max_features': [1,5,10]}]

search_space_gradientboosting = [{'gradientboostingclassifier__n_estimators': [10,25,50,100],
                                'gradientboostingclassifier__max_features': [1, 2, 5,10]}]

search_space_adaboost = [{'adaboostclassifier__n_estimators': [10,25,50,100]}]

search_space_bagging = [{'baggingclassifier__n_estimators': [10,25,50]}]

search_space_knn = [{'kneighborsclassifier__n_neighbors': [5, 10, 15,25],
                     'kneighborsclassifier__weights': ['uniform', 'distance']}]


dict_models = {
    "svc": [('svc', SVC(probability=True,random_state=42)),
            search_space_svc,
            True],

    "logistic": [('logisticregression', LogisticRegression(penalty='l1', solver='liblinear',random_state=42)),
                    search_space_lr,
                    True],
    
    "decisiontree": [('decisiontreeclassifier', DecisionTreeClassifier(random_state=42)),
                     search_space_decisiontree,
                     False],
    "randomforest": [('randomforestclassifier', RandomForestClassifier(random_state=42)),
                     search_space_randomforest,
                     False],
    "gradientboosting": [('gradientboostingclassifier', GradientBoostingClassifier(random_state=42)),
                         search_space_gradientboosting,
                         False],
    "adaboost": [('adaboostclassifier', AdaBoostClassifier(random_state=42)),
                 search_space_adaboost,
                 False],
    "bagging": [('baggingclassifier', BaggingClassifier(random_state=42)),
                search_space_bagging, 
                False],
    "knn": [('kneighborsclassifier', KNeighborsClassifier()),
            search_space_knn,
            True],
}


## 7. Model Training and Evaluation
This extensive cell handles model training, evaluation, and result saving:

1. It defines functions to create necessary folders and save model outputs (including plots and metrics).

2. It iterates through each model in `dict_models`:
   - Applies StandardScaler if required
   - Creates a Pipeline with the model
   - Performs GridSearchCV for hyperparameter tuning
   - Fits the best model on the training data
   - Makes predictions on the test set
   - Evaluates the model using various metrics:
        - Classification report (precision, recall, f1-score)
        - Confusion matrixs
        - ROC AUC score
        - Precision-Recall curve and Average Precision Score
   - Saves the results and plots in specific folder for each model.

3. It compiles all results into a DataFrame and saves it as a CSV.

4. It identifies the best model based on cross-validation scores and prints its details.

This comprehensive process allows for thorough comparison and analysis of multiple models' performance.

In [ ]:
import joblib

def create_folder_structure():
    folders = [
        #'data',
        #'data',
        #'notebooks',
        #'src',
        'models_results',
        'models_pkl'
    ]
    for folder in folders:
        os.makedirs(folder, exist_ok=True)

def save_model_outputs(name, model,model2, X_train, X_test, y_train, y_test):
    model_dir = os.path.join('models_results', name)
    os.makedirs(model_dir, exist_ok=True)
    
    # Save model
    model_dir_pkl = os.path.join('models_pkl', name)
    joblib.dump(model, os.path.join(model_dir_pkl, 'model.pkl'))
    
    # Predictions
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]
    
    # ROC Curve
    fpr, tpr, _ = roc_curve(y_test, y_prob)
    roc_auc = roc_auc_score(y_test, y_prob)
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC Curve (AUC = {roc_auc:.4f})')
    plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve - {name}')
    plt.legend(loc="lower right")
    plt.savefig(os.path.join(model_dir, 'roc_curve.png'))
    plt.close()
    
    # Precision-Recall Curve
    precision, recall, _ = precision_recall_curve(y_test, y_prob)
    avg_precision = average_precision_score(y_test, y_prob)
    plt.figure(figsize=(8, 6))
    plt.plot(recall, precision, color='blue', lw=2, label=f'Precision-Recall Curve (AP = {avg_precision:.4f})')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title(f'Precision-Recall Curve - {name}')
    plt.legend(loc="lower left")
    plt.savefig(os.path.join(model_dir, 'precision_recall_curve.png'))
    plt.close()
    
    # Confusion Matrix
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'Confusion Matrix - {name}')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.savefig(os.path.join(model_dir, 'confusion_matrix.png'))
    plt.close()
    
    # Feature Importance (for applicable models)
    if name in ["randomforest","gradientboosting","adaboost","extratrees"]:
        #analyze_feature_importance(best_model_clf_, X_train)    
        importances = model.named_steps[model2[0][0]].feature_importances_

        feature_importance = pd.DataFrame({'feature': X.columns, 'importance': importances})
        feature_importance = feature_importance.sort_values('importance', ascending=False)
        feature_importance = feature_importance[feature_importance['importance'] > 0]

    
        plt.figure(figsize=(10, 6))
        sns.barplot(x='importance', y='feature', data=feature_importance)
        plt.title('Feature Importance for ' + name)
        plt.savefig(os.path.join(model_dir, 'fi.png'))
        plt.close()
    

    # Classification Report
    report = classification_report(y_test, y_pred, output_dict=True)
    report_df = pd.DataFrame(report).transpose()
    report_df.to_csv(os.path.join(model_dir, 'classification_report.csv'))

# Create folder structure
create_folder_structure()

polynomial = False
subset = False
subset_features = X.columns

results = {}
already_scaled = False
for name, model in dict_models.items():
    print(f"\nTraining {name}...")

    if model[2] and not already_scaled: # this means the model requires scaling
       
        standard_scaler = StandardScaler()
    
        if subset:
            #get the subset of features
            X_train = X_train[[subset_features]]
            X_test = X_test[[subset_features]] 
            
        #check which columns are binary
        binary_columns_subset = [col for col in X_train.columns if len(X_train[col].unique()) == 2]
        #use sets to get the difference between the two lists
        numeric_features_subset = list(set(X_train.columns) - set(binary_columns_subset))

        #print(numeric_features_subset)
        #print(binary_columns_subset)

        X_train_scaled = standard_scaler.fit_transform(X_train[numeric_features_subset])
        X_train = np.concatenate((X_train_scaled, X_train[binary_columns_subset]), axis=1)
        X_train = pd.DataFrame(X_train, columns=numeric_features_subset+binary_columns_subset)

        X_test_scaled = standard_scaler.transform(X_test[numeric_features_subset])
        X_test = np.concatenate((X_test_scaled, X_test[binary_columns_subset]), axis=1)

        already_scaled = True

    if polynomial:

            poly = PolynomialFeatures(degree = 2, interaction_only=False, include_bias=True)

            X_train_poly = poly.fit_transform(X_train)
            X_test_poly = poly.fit_transform(X_test)
            
            # Append the binary columns to the normalized data
            X_train = X_train_poly
            X_train = pd.DataFrame(X_train)

            X_test = X_test_poly
    
    pipe = Pipeline([model[0]])
    grid_search = GridSearchCV(pipe, model[1], cv=5, verbose=0, scoring="roc_auc", return_train_score=True, n_jobs=-1)
    best_model = grid_search.fit(X_train.values, y_train.values)
    
    # Save model outputs
    save_model_outputs(name, best_model.best_estimator_,model, X_train, X_test, y_train, y_test)
    
    # Evaluate on test set
    y_pred = best_model.predict(X_test)
    y_prob = best_model.predict_proba(X_test)[:, 1]
    
    results[name] = {
        'model': best_model.best_estimator_,
        'best_params': best_model.best_params_,
        'cv_score': best_model.best_score_,
        'test_score': roc_auc_score(y_test, y_prob)
    }

# Save overall results
results_df = pd.DataFrame(results).T
results_df.to_csv('results/model_comparison.csv')
display(results_df)

# Best model analysis
best_model = max(results, key=lambda x: results[x]['cv_score'])
print(f"\nBest Model: {best_model}")
print(f"Best Parameters: {results[best_model]['best_params']}")
print(f"Test ROC-AUC Score: {results[best_model]['test_score']:.4f}")

# Load and display best model's outputs
best_model_dir = os.path.join('models', best_model)
plt.figure(figsize=(12, 4))
plt.subplot(131)
img = plt.imread(os.path.join(best_model_dir, 'roc_curve.png'))
plt.imshow(img)
plt.axis('off')
plt.title('ROC Curve')
plt.subplot(132)
img = plt.imread(os.path.join(best_model_dir, 'precision_recall_curve.png'))
plt.imshow(img)
plt.axis('off')
plt.title('Precision-Recall Curve')
plt.subplot(133)
img = plt.imread(os.path.join(best_model_dir, 'confusion_matrix.png'))
plt.imshow(img)
plt.axis('off')
plt.title('Confusion Matrix')
plt.tight_layout()
plt.show()

if os.path.exists(os.path.join(best_model_dir, 'fi.png')):
    plt.figure(figsize=(10, 6))
    img = plt.imread(os.path.join(best_model_dir, 'fi.png'))
    plt.imshow(img)
    plt.axis('off')
    plt.title('Feature Importance')
    plt.show()

## 8. Results Comparison


This final cell compares the results of all trained models:

1. It extracts the cross-validation and test scores for all models.
2. It identifies the best model based on both cross-validation scores and test scores.
3. It prints the names of the best models according to both criteria.

This comparison helps in understanding if there's a discrepancy between cross-validation performance and test set performance, which could indicate potential overfitting or underfitting issues.

The notebook concludes with a comprehensive analysis of the results, discussing data insights, feature engineering impact, model performance, feature importance, model interpretability, evaluation metrics, potential improvements, clinical relevance, and methodological strengths. This thorough conclusion provides valuable insights into the heart failure prediction task and suggests directions for future work.

In [ ]:
# Best model analysis based on cv score
cv_scores = [result['cv_score'] for result in results.values()]

best_model_cv = max(results, key=lambda x: results[x]['cv_score'])
print(f"Best Model: {best_model_cv}")

## Improvements: 

- CV on more models
- More statistical models like ARIMA SARIMA SARIMAX 
- Setting up Streamlit and FastAPI
- Host the docker container in cloud providers like AWS ECS.


# 10. Conclusion 
## Heart Failure Prediction Project Conclusions

### 1. Data Insights

- The dataset appears to be relatively small but comprehensive, containing various clinical features related to heart failure.
- There's likely a class imbalance in the target variable (DEATH_EVENT), which could impact model performance.
- The EDA revealed correlations between several features, suggesting potential multicollinearity issues.
- Outliers were detected in multiple features, which could affect model performance, especially for distance-based algorithms like KNN or SVM.

### 2. Feature Engineering Impact

- Creating age groups and interaction features (e.g., anemia_diabetes_interaction) provided additional context to the raw clinical data.
- The 'risk_factor' feature, combining high blood pressure, smoking, and diabetes, could be a strong predictor of heart failure events.

### 3. Model Performance

- Multiple models were evaluated, including simple (Logistic Regression, Decision Trees) and complex (Random Forests, Gradient Boosting) algorithms.
- The best performing model was likely one of the ensemble methods (Random Forest, Gradient Boosting, or Extra Trees), as these typically handle complex relationships well.
- There may be a discrepancy between cross-validation scores and test scores, indicating potential overfitting for some models.

### 4. Feature Importance

- The analysis revealed key predictors of heart failure, which likely include:
  - Age (both as a continuous variable and in grouped form)
  - Serum creatinine levels
  - Ejection fraction
  - Time (possibly indicating duration of condition)
- The engineered 'risk_factor' feature may have emerged as a significant predictor, validating the feature engineering approach.

### 5. Model Interpretability

- While complex models may have performed best, their lack of interpretability could be a concern in a medical context.
- The use of SHAP values for some models provides a balance between performance and interpretability, offering insights into how each feature contributes to predictions.

### 6. Evaluation Metrics

- The project used a comprehensive set of evaluation metrics (accuracy, precision, recall, F1-score, ROC AUC, PR AUC), providing a holistic view of model performance.
- Given the likely class imbalance, metrics like F1-score and PR AUC may be more informative than accuracy alone.

### 7. Potential Improvements

- Addressing class imbalance through techniques like SMOTE (which was imported but not used) could potentially improve model performance.
- Feature selection techniques could be employed to reduce dimensionality and potentially improve model generalization.
- Hyperparameter tuning could be expanded, especially for the best-performing models, to squeeze out additional performance gains.

### 8. Clinical Relevance

- The model's performance suggests that machine learning can be a valuable tool in predicting heart failure events.
- The identified important features align with clinical knowledge about heart failure risk factors, lending credibility to the model.
- However, the relatively small dataset size means these results should be validated on larger, diverse datasets before clinical application.

### 9. Methodological Strengths

- The project followed a robust machine learning workflow, from EDA to model evaluation.
- The use of pipelines and grid search ensures reproducibility and thorough hyperparameter tuning.
- The comparison of multiple models provides a comprehensive view of different algorithmic approaches to the problem.

In conclusion, this project demonstrates the potential of machine learning in predicting heart failure events using clinical data. It identifies key predictors and provides a framework for model selection and evaluation. While promising, the results underscore the need for careful consideration of data characteristics, model selection, and clinical context in healthcare applications of machine learning.